# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 10:42AM,239322,21,399280391,"NBTY Global, Inc.",Released
1,Jun 10 2022 10:31AM,239321,10,8122606,Eywa Pharma Inc.,Released
2,Jun 10 2022 10:31AM,239321,10,8122607,Eywa Pharma Inc.,Released
3,Jun 10 2022 10:31AM,239321,10,8123236,Eywa Pharma Inc.,Released
4,Jun 10 2022 10:31AM,239321,10,8123239-PR,Eywa Pharma Inc.,Released
5,Jun 10 2022 10:31AM,239321,10,8123240,Eywa Pharma Inc.,Released
6,Jun 10 2022 10:31AM,239321,10,8123241,Eywa Pharma Inc.,Released
7,Jun 10 2022 10:31AM,239321,10,8123768,Eywa Pharma Inc.,Released
8,Jun 10 2022 10:31AM,239321,10,8123769,Eywa Pharma Inc.,Released
9,Jun 10 2022 10:24AM,239320,21,399290068,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,239318,Released,11
26,239319,Released,9
27,239320,Released,1
28,239321,Released,8
29,239322,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239318,NaN,NaN,11.0
239319,NaN,NaN,9.0
239320,NaN,NaN,1.0
239321,NaN,NaN,8.0
239322,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239253,0.0,0.0,1.0
239258,0.0,0.0,1.0
239261,0.0,1.0,1.0
239271,1.0,23.0,4.0
239286,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239253,0,0,1
239258,0,0,1
239261,0,1,1
239271,1,23,4
239286,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239253,0,0,1
1,239258,0,0,1
2,239261,0,1,1
3,239271,1,23,4
4,239286,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239253,,,1
1,239258,,,1
2,239261,,1,1
3,239271,1,23,4
4,239286,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc."
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.
9,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc."
10,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions
19,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions
30,Jun 10 2022 9:45AM,239316,19,"AdvaGen Pharma, Ltd"
38,Jun 10 2022 9:28AM,239315,10,ISDIN Corporation
67,Jun 10 2022 9:25AM,239313,20,Alumier Labs Inc.
87,Jun 10 2022 8:43AM,239305,15,"Person & Covey, Inc."
116,Jun 10 2022 8:30AM,239306,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",,,1
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.,,,8
2,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",,,1
3,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,,,9
4,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions,,,11
5,Jun 10 2022 9:45AM,239316,19,"AdvaGen Pharma, Ltd",,4,4
6,Jun 10 2022 9:28AM,239315,10,ISDIN Corporation,1,12,16
7,Jun 10 2022 9:25AM,239313,20,Alumier Labs Inc.,,,20
8,Jun 10 2022 8:43AM,239305,15,"Person & Covey, Inc.",,12,17
9,Jun 10 2022 8:30AM,239306,12,Hush Hush,,11,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",1,,
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.,8,,
2,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,9,,
4,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions,11,,
5,Jun 10 2022 9:45AM,239316,19,"AdvaGen Pharma, Ltd",4,4,
6,Jun 10 2022 9:28AM,239315,10,ISDIN Corporation,16,12,1
7,Jun 10 2022 9:25AM,239313,20,Alumier Labs Inc.,20,,
8,Jun 10 2022 8:43AM,239305,15,"Person & Covey, Inc.",17,12,
9,Jun 10 2022 8:30AM,239306,12,Hush Hush,,11,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",1,,
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.,8,,
2,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",1,,
3,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,9,,
4,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.,8.0,NaN,NaN
2,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,9.0,NaN,NaN
4,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 10:42AM,239322,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 10 2022 10:31AM,239321,10,Eywa Pharma Inc.,8.0,0.0,0.0
2,Jun 10 2022 10:24AM,239320,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 10 2022 10:08AM,239319,16,Sartorius Bioprocess Solutions,9.0,0.0,0.0
4,Jun 10 2022 10:03AM,239318,16,Sartorius Bioprocess Solutions,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,478636,24.0,12.0,1.0
12,478567,1.0,12.0,0.0
15,957214,28.0,16.0,0.0
16,717895,21.0,0.0,0.0
18,239253,1.0,0.0,0.0
19,478606,5.0,4.0,0.0
20,478584,24.0,23.0,1.0
21,1435805,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,478636,24.0,12.0,1.0
1,12,478567,1.0,12.0,0.0
2,15,957214,28.0,16.0,0.0
3,16,717895,21.0,0.0,0.0
4,18,239253,1.0,0.0,0.0
5,19,478606,5.0,4.0,0.0
6,20,478584,24.0,23.0,1.0
7,21,1435805,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,24.0,12.0,1.0
1,12,1.0,12.0,0.0
2,15,28.0,16.0,0.0
3,16,21.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,5.0,4.0,0.0
6,20,24.0,23.0,1.0
7,21,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,12,Released,1.0
2,15,Released,28.0
3,16,Released,21.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,12.0,12.0,16.0,0.0,0.0,4.0,23.0,0.0
Released,24.0,1.0,28.0,21.0,1.0,5.0,24.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,12.0,12.0,16.0,0.0,0.0,4.0,23.0,0.0
2,Released,24.0,1.0,28.0,21.0,1.0,5.0,24.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,12.0,12.0,16.0,0.0,0.0,4.0,23.0,0.0
2,Released,24.0,1.0,28.0,21.0,1.0,5.0,24.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()